In [1]:
%python
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like #datareader problem probably fixed in next version of datareader
from pandas_datareader import data as pdr
import datetime
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as sf
from pyspark.sql.window import Window
import numpy as np
import fix_yahoo_finance as yf
from math import log

In [2]:
%python
yf.pdr_override() # <== that's all it takes :-)


start_date=datetime.date(1980, 12, 12)
end_date= datetime.date(2018, 1, 1)

stock_list = ["AAPL"]

stock_str = ""
for i in range(len(stock_list)):
    stock_str  = stock_str + stock_list[i] + "."

main_df = pd.DataFrame()

for stock in range(len(stock_list)):
     df = pdr.get_data_yahoo(stock_list[stock], start=start_date, end=end_date)
     #df.drop(['Close','High', 'Low' , 'Open', 'Volume'], axis=1, inplace=True)
     df.rename(columns={'Adj Close': 'Adj_close'}, inplace=True)
     if main_df.empty:
         main_df = df
     else:
        main_df = main_df.join(df) 
           
#main_df["Date"]=main_df.index
main_df.reset_index(level=0, inplace=True)

[ 0% ][*********************100%***********************] 1 of 1 downloaded

In [3]:
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
spark = SparkSession(sc)

In [4]:
df = spark.createDataFrame(df)
df = df.withColumn("ticker", sf.lit("AAPL"))
df.show(10)

+-------------------+--------+--------+-------------------+-------------------+--------------------+---------+------+
 Date| Open| High| Low| Close| Adj_close| Volume|ticker|
+-------------------+--------+--------+-------------------+-------------------+--------------------+---------+------+
1980-12-12 00:00:00|0.513393|0.515625| 0.513393| 0.513393| 0.023007|117258400| AAPL|
1980-12-15 00:00:00|0.488839|0.488839|0.48660699999999996|0.48660699999999996| 0.021807| 43971200| AAPL|
1980-12-16 00:00:00|0.453125|0.453125|0.45089300000000004|0.45089300000000004|0.020206000000000002| 26432000| AAPL|
1980-12-17 00:00:00|0.462054|0.464286| 0.462054| 0.462054| 0.020706| 21610400| AAPL|
1980-12-18 00:00:00|0.475446|0.477679| 0.475446| 0.475446| 0.021307| 18362400| AAPL|
1980-12-19 00:00:00|0.504464|0.506696| 0.504464| 0.504464| 0.022607| 12157600| AAPL|
1980-12-22 00:00:00|0.529018| 0.53125| 0.529018| 0.529018| 0.023707| 9340800| AAPL|
1980-12-23 00:00:00|0.551339|0.553571| 0.551339| 0.551339| 0.024708| 11737600| AAPL|
1980-12-24 00:00:00|0.580357|0.582589| 0.580357| 0.580357| 0.026008| 12000800| AAPL|
1980-12-26 00:00:00|0.633929|0.636161| 0.633929| 0.633929|0.028408999999999997| 13893600| AAPL|
+-------------------+--------+--------+-------------------+-------------------+--------------------+---------+------+
only showing top 10 rows

In [5]:
w = Window().partitionBy("ticker").orderBy("date")
df = df.withColumn("Log_Adj_Close", sf.log("Adj_close"))

#Returns
df = df.withColumn("log_return", -1* (sf.col("Log_Adj_Close") - sf.lag("Log_Adj_Close", 1).over(w)) / sf.lag("Log_Adj_Close", 5).over(w))
df = df.withColumn("weekly_log_return", -1* (sf.col("Log_Adj_Close") - sf.lag("Log_Adj_Close", 5).over(w)) / sf.lag("Log_Adj_Close", 10).over(w))
df = df.withColumn("biweekly_log_return", -1* (sf.col("Log_Adj_Close") - sf.lag("Log_Adj_Close", 10).over(w)) / sf.lag("Log_Adj_Close", 21).over(w))
df = df.withColumn("monthly_log_return", -1* (sf.col("Log_Adj_Close") - sf.lag("Log_Adj_Close", 21).over(w)) / sf.lag("Log_Adj_Close", 42).over(w))
df = df.withColumn("bimonthly_log_return", -1* (sf.col("Log_Adj_Close") - sf.lag("Log_Adj_Close", 42).over(w)) / sf.lag("Log_Adj_Close", 126).over(w))
df = df.withColumn("annual_log_return", -1* (sf.col("Log_Adj_Close") - sf.lag("Log_Adj_Close", 252).over(w)) / sf.lag("Log_Adj_Close", 252).over(w))

#Volume
df = df.withColumn("daily_volume_diff", (sf.col("Volume") - sf.lag("Volume", 1).over(w))) 
df = df.withColumn("weekly_volume_diff", (sf.col("Volume") - sf.lag("Volume", 5).over(w))) 
df = df.withColumn("biweekly_volume_diff", (sf.col("Volume") - sf.lag("Volume", 10).over(w))) 
df = df.withColumn("monthly_volume_diff", (sf.col("Volume") - sf.lag("Volume", 21).over(w))) 
df = df.withColumn("bimonthly_volume_diff", (sf.col("Volume") - sf.lag("Volume", 42).over(w))) 
df = df.withColumn("annual_volume_diff", -1* (sf.col("Volume") - sf.lag("Volume", 252).over(w))) 

#Everything else
w = Window.orderBy('Date').rowsBetween(-5, 0)
df = df.withColumn("weekly_mean", sf.avg('log_return').over(w))
df = df.withColumn("weekly_std", sf.stddev('log_return').over(w))
df = df.withColumn("weekly_volume_mean", sf.avg('Volume').over(w))
df = df.withColumn("weekly_volume_std", sf.stddev('Volume').over(w))

w = Window.orderBy('Date').rowsBetween(-10, 0)
df = df.withColumn("biweekly_mean", sf.avg('log_return').over(w))
df = df.withColumn("biweekly_std", sf.stddev('log_return').over(w))
df = df.withColumn("biweekly_volume_mean", sf.avg('Volume').over(w))
df = df.withColumn("biweekly_volume_std", sf.stddev('Volume').over(w))

w = Window.orderBy('Date').rowsBetween(-21, 0)
df = df.withColumn("monthly_mean", sf.avg('log_return').over(w))
df = df.withColumn("monthly_std", sf.stddev('log_return').over(w))
df = df.withColumn("monthly_volume_mean", sf.avg('Volume').over(w))
df = df.withColumn("monthly_volume_std", sf.stddev('Volume').over(w))

w = Window.orderBy('Date').rowsBetween(-42, 0)
df = df.withColumn("bimonthly_mean", sf.avg('log_return').over(w))
df = df.withColumn("bimonthly_std", sf.stddev('log_return').over(w))
df = df.withColumn("bimonthly_volume_mean", sf.avg('Volume').over(w))
df = df.withColumn("bimonthly_volume_std", sf.stddev('Volume').over(w))

w = Window.orderBy('Date').rowsBetween(-126, 0)
df = df.withColumn("semiannual_mean", sf.avg('log_return').over(w))
df = df.withColumn("semiannual_std", sf.stddev('log_return').over(w))
df = df.withColumn("semiannual_volume_mean", sf.avg('Volume').over(w))
df = df.withColumn("semiannual_volume_std", sf.stddev('Volume').over(w))

w = Window.orderBy('Date').rowsBetween(-252, 0)
df = df.withColumn("annual_mean", sf.avg('log_return').over(w))
df = df.withColumn("annual_std", sf.stddev('log_return').over(w))
df = df.withColumn("annual_volume_mean", sf.avg('Volume').over(w))
df = df.withColumn("annual_volume_std", sf.stddev('Volume').over(w))

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer

df2 = df.select('Date', date_format('Date', 'u').alias('dow'), date_format('Date', 'd').alias('day'), date_format('Date', 'D').alias('doy'), date_format('Date', 'W').alias('wom'), date_format('Date', 'L').alias('moy'), date_format('Date', 'w').alias('woy')) #, date_format('Date', 'E').alias('day'))
df3=df2.orderBy("Date")


indexers = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(df3.columns)-set(["Date"]))]
one_hot = [OneHotEncoder(inputCol= column +"_index", outputCol= column + "_one_hot") for column in list(set(df3.columns)-set(["Date"]))]

pipeline = Pipeline(stages=indexers +one_hot )
df_r = pipeline.fit(df3).transform(df3)

df_final = df.join(df_r, df.Date == df_r.Date)
df_final.show()

+-------------------+------------------+------------------+------------------+-------------------+--------------------+--------+------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------------+--------------------+-------------------+---------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+----------------------+---------------------+--------------------+--------------------+--------------------+--------------------+-------------------+---+---+---+---+---+---+---------+---------+---------+---------+---------+---------+---------------+-------------+---------------+---------------+-------------+-----------------+
 Date| Open| High| Low| Close| Adj_close| Volume|ticker| Log_Adj_Close| log_return| weekly_log_return| biweekly_log_return| monthly_log_return|bimonthly_log_return| annual_log_return|daily_volume_diff|weekly_volume_diff|biweekly_volume_diff|monthly_volume_diff|bimonthly_volume_diff|annual_volume_diff| weekly_mean| weekly_std| weekly_volume_mean| weekly_volume_std| biweekly_mean| biweekly_std|biweekly_volume_mean| biweekly_volume_std| monthly_mean| monthly_std| monthly_volume_mean| monthly_volume_std| bimonthly_mean| bimonthly_std|bimonthly_volume_mean|bimonthly_volume_std| semiannual_mean| semiannual_std|semiannual_volume_mean|semiannual_volume_std| annual_mean| annual_std| annual_volume_mean| annual_volume_std| Date|dow|day|doy|wom|moy|woy|woy_index|wom_index|moy_index|day_index|dow_index|doy_index| woy_one_hot| wom_one_hot| moy_one_hot| day_one_hot| dow_one_hot| doy_one_hot|
+-------------------+------------------+------------------+------------------+-------------------+--------------------+--------+------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------------+--------------------+-------------------+---------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+----------------------+---------------------+--------------------+--------------------+--------------------+--------------------+-------------------+---+---+---+---+---+---+---------+---------+---------+---------+---------+---------+---------------+-------------+---------------+---------------+-------------+-----------------+
1981-07-21 00:00:00| 0.430804| 0.430804| 0.428571| 0.428571| 0.019206| 7985600| AAPL| -3.952532548766356|-0.00131042076117...|0.002679476868799...|-0.01230694753361...|-0.06113978914142...|-0.04564222597885861| null| 2072000| 3040800| 4026400| 1108800| 4664800| null|0.002314121879182947|0.010537820452079577| 5057733.333333333| 1803837.858197534|-8.25070605430112...|0.012553663700892134| 6537236.363636363| 3481245.999716977|-0.00314891029150...|0.010391645566701383| 7680400.0| 8297177.9363025995|-8.73151012209287E-4|0.009313707068578191| 1.1050102325581396E7| 9622571.962330682|-6.93394197428355...|0.008635846512689645| 7607313.385826772| 8031397.294609888|-2.37613464613684...|0.008982584073939306| 9143105.263157895|1.2167723713983634E7|1981-07-21 00:00:00| 2| 21|202| 4| 7| 30| 23.0| 2.0| 5.0| 16.0| 1.0| 24.0|(52,[23],[1.0])|(5,[2],[1.0])| (11,[5],[1.0])|(30,[16],[1.0])|(4,[1],[1.0])| 